In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import random

In [32]:
# 设置随机种子以确保结果可重复
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [62]:
# 生成合成数据
X, y = make_classification(n_samples=100,    # 总样本数
                           n_features=20,     # 特征数
                           n_informative=2,   # 有信息的特征数
                           n_redundant=10,    # 冗余特征数
                           n_clusters_per_class=1,
                           weights=[0.9, 0.1],# 类别不平衡，90% 类别0，10% 类别1
                           flip_y=0,          # 不引入标签噪声
                           random_state=42)

# 分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 转换为 PyTorch 张量
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# 创建 DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [63]:
# 定义 PolyLoss
class PolyLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, beta=1, k=3, reduction='mean'):
        super(PolyLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.beta = beta
        self.k = k
        self.reduction = reduction

    def forward(self, inputs, targets):
        # 计算交叉熵损失
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
        # 计算预测概率
        pt = torch.exp(-ce_loss)
        
        # PolyLoss 公式
        poly_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss + self.beta * (1 - pt) ** self.k
        
        if self.reduction == 'mean':
            return poly_loss.mean()
        elif self.reduction == 'sum':
            return poly_loss.sum()
        else:
            return poly_loss

In [64]:
# 定义简单的神经网络
class SimpleNN(nn.Module):
    def __init__(self, input_size=20, hidden_size=16, num_classes=2):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [65]:
# 定义训练函数
def train_model(model, criterion, optimizer, train_loader, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    return model

In [66]:
# 定义评估函数
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    print(classification_report(all_targets, all_preds, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(all_targets, all_preds))

In [67]:
# 训练并评估使用普通交叉熵损失的模型
model_ce = SimpleNN()
criterion_ce = nn.CrossEntropyLoss()
optimizer_ce = torch.optim.Adam(model_ce.parameters(), lr=0.001)

model_ce = train_model(model_ce, criterion_ce, optimizer_ce, train_loader, num_epochs=10)
print("测试集性能（交叉熵损失）:")
evaluate_model(model_ce, test_loader)

Epoch [1/10], Loss: 0.5619
Epoch [2/10], Loss: 0.5327
Epoch [3/10], Loss: 0.5044
Epoch [4/10], Loss: 0.4787
Epoch [5/10], Loss: 0.4536
Epoch [6/10], Loss: 0.4300
Epoch [7/10], Loss: 0.4083
Epoch [8/10], Loss: 0.3868
Epoch [9/10], Loss: 0.3665
Epoch [10/10], Loss: 0.3472
测试集性能（交叉熵损失）:
              precision    recall  f1-score   support

           0     0.9000    1.0000    0.9474        18
           1     0.0000    0.0000    0.0000         2

    accuracy                         0.9000        20
   macro avg     0.4500    0.5000    0.4737        20
weighted avg     0.8100    0.9000    0.8526        20

Confusion Matrix:
[[18  0]
 [ 2  0]]


D:\python_etc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\python_etc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\python_etc\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
# 训练并评估使用 PolyLoss 的模型
print("\n=== 使用 PolyLoss ===")
model_poly = SimpleNN()
criterion_poly = PolyLoss(alpha=1, gamma=2, beta=1, k=3)
optimizer_poly = torch.optim.Adam(model_poly.parameters(), lr=0.001)

model_poly = train_model(model_poly, criterion_poly, optimizer_poly, train_loader, num_epochs=10)
print("测试集性能（PolyLoss）:")
evaluate_model(model_poly, test_loader)


=== 使用 PolyLoss ===
Epoch [1/10], Loss: 0.3351
Epoch [2/10], Loss: 0.3062
Epoch [3/10], Loss: 0.2798
Epoch [4/10], Loss: 0.2541
Epoch [5/10], Loss: 0.2309
Epoch [6/10], Loss: 0.2081
Epoch [7/10], Loss: 0.1881
Epoch [8/10], Loss: 0.1691
Epoch [9/10], Loss: 0.1528
Epoch [10/10], Loss: 0.1377
测试集性能（PolyLoss）:
              precision    recall  f1-score   support

           0     0.9474    1.0000    0.9730        18
           1     1.0000    0.5000    0.6667         2

    accuracy                         0.9500        20
   macro avg     0.9737    0.7500    0.8198        20
weighted avg     0.9526    0.9500    0.9423        20

Confusion Matrix:
[[18  0]
 [ 1  1]]
